In [1]:
import numpy as np
import  tensorflow as tf


In [2]:
q = tf.FIFOQueue(3,"float")
init = q.enqueue_many(([0.1,0.2,0.3],))

x = q.dequeue()
y = x + 1 
q_inc = q.enqueue([y])

with tf.Session() as sess:
    sess.run(init)
    quelen = sess.run(q.size())
    for i in range(2):
        sess.run(q_inc)
    quelen = sess.run(q.size())
    for i in range(quelen):
        print(sess.run(q.dequeue()))

0.3
1.1
1.2


In [3]:
q = tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes="float")
sess = tf.Session()
for i in range(10):
    sess.run(q.enqueue(i))
quelen = sess.run(q.size())

for i in range(0,8):
    print(sess.run(q.dequeue()))
    
#设置会话在运行时的等待时间来消除阻断
# run_options = tf.RunOptions(timeout_in_ms = 10000) # 等待 10 秒
# try:   
#     sess.run(q.dequeue(), options=run_options)
# except tf.errors.DeadlineExceededError:
#     print('out of range')

1.0
9.0
3.0
7.0
4.0
6.0
8.0
0.0


#队列管理器

In [ ]:
q = tf.FIFOQueue(1000, "float")
counter = tf.Variable(0.0) # 计数器
increment_op = tf.assign_add(counter, tf.constant(1.0)) # 操作：给计数器加 1
enqueue_op = q.enqueue(counter) # 操作：计数器值加入队列
qr = tf.train.QueueRunner(q, enqueue_ops=[increment_op, enqueue_op] * 1)
#主线程
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    eneque_threads = qr.create_threads(sess,start=True)
    for i in range(10):
        print(sess.run(q.dequeue()))

#线程与协调器


In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
# Coordinator：协调器，协调线程间的关系可以视为一种信号量，用来做同步
coord = tf.train.Coordinator()
# 启动入队线程，协调器是线程的参数
enqueue_threads = qr.create_threads(sess, coord = coord,start=True)
# 主线程
for i in range(0, 10):
    print(sess.run(q.dequeue()))
coord.request_stop()# 通知其他线程关闭
coord.join(enqueue_threads) # join 操作等待其他线程结束，其他所有
# 线程关闭之后，这一函数才能返回
# coord.request_stop()
# # 主线程
# for i in range(0, 10):
#     try:
#         print(sess.run(q.dequeue()))
#     except tf.errors.OutOfRangeError:
#         break
# coord.join(enqueue_threads)

3.0
14.0
22.0
36.0
46.0
56.0
69.0
78.0
91.0
105.0


#4.10 加载数据
预加载数据（preloaded data）：在 TensorFlow 图中定义常量或变量来保存所有数据。
● 填充数据（feeding）：Python 产生数据，再把数据填充后端。
● 从文件读取数据（reading from file）：从文件中直接读取，让队列管理器从文件中读取
数据。

In [20]:
import tensorflow as tf
# 设计图
a1 = tf.placeholder(tf.int16)
a2 = tf.placeholder(tf.int16)
b = tf.add(a1, a2)
# 用 Python 产生数据
li1 = [2, 3, 4]
li2 = [4, 0, 1]
# 打开一个会话，将数据填充给后端
with tf.Session() as sess:
    print (sess.run(b, feed_dict={a1: li1, a2: li2}))

[6 3 5]


NameError: name 'mnist' is not defined